In [1]:
import pandas as pd
import os


In [2]:
beer_types = pd.read_csv("Beer_Styles.csv")
beer_types.head()

,beer_style,subgroup
0,Hefeweizen,German Wheat Beer
1,English Strong Ale,Strong Ale
2,Foreign / Export Stout,Stout
3,German Pilsener,Standard American Beer
4,American Double / Imperial IPA,IPA


In [3]:
df = pd.read_csv("beer_reviews.csv")
parsed_data = df.drop_duplicates()
parsed_data = parsed_data.dropna()
parsed_data = parsed_data.reset_index()
parsed_data.head()

,index,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [4]:
for name in beer_types.index:
    parsed_data = parsed_data.replace(to_replace=beer_types.iloc[name]['beer_style'], value=beer_types.loc[name]['subgroup'])


In [5]:
parsed_data.head()

,index,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,German Wheat Beer,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,Standard American Beer,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [ ]:
data = parsed_data[["review_overall", "review_aroma", "review_appearance", "review_palate","review_taste", "beer_abv"]]
feature_names = data.columns
data.head()

In [ ]:
target = parsed_data[["beer_style"]]
target_names = df["beer_style"].unique()
target_names